In [1]:
# Requirements
import csv
import numpy
#import nltk
#from nltk.corpus import stopwords
#from nltk.collocations import *
from datetime import datetime
#import requests
#from bs4 import BeautifulSoup
#import re
#import fnmatch

# EDGI's web monitoring scripts
#!git clone https://github.com/edgi-govdata-archiving/wayback #&>/dev/null; 
!pip install wayback #&>/dev/null;
#!python wayback/setup.py develop #&>/dev/null;

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from wayback import WaybackClient

/Users/enost/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [33]:
with WaybackClient() as client:
       dump = client.search("epa.gov", from_date=datetime(dates[0], dates[1],dates[2]), to_date=datetime(dates[3], dates[4], dates[5])) # list_versions calls the CDX API from internetarchive.py from the webmonitoring repo
       versions = reversed(list(dump))
       for version in versions: # For each version in all the snapshots
           #print(version.status_code)
           if version.status_code == 200 or version.status_code == '-': # If the IA snapshot was viable...
              url=version.raw_url
              print(url)
              response = client.get_memento(version.raw_url)
              content = response.content.decode()
              for term in terms:
                  count = content.lower().count('climate change')
                  print (count)
              break

http://web.archive.org/web/20160630065006id_/https://www3.epa.gov/
3
http://web.archive.org/web/20160629190059id_/https://www3.epa.gov/
3
http://web.archive.org/web/20160628171642id_/https://www3.epa.gov/
3
http://web.archive.org/web/20160628160750id_/https://www3.epa.gov/


KeyboardInterrupt: 

In [42]:
# Parameters
terms = ['climate', 'climate change'] # The terms we want to count
dates = [2016,1,1, 2016,7,1] # Looking for snapshots between Jan 1 2016 and July 1 2016, working backwards.
pages = ['epa.gov', 'epa.gov/climatechange'] # The pages we want to look at

In [51]:
final_urls= {}
row_count = len(pages)
column_count = len(terms) 
matrix = numpy.full((row_count,column_count), 999, dtype=numpy.int16) # Default is 999 until counted otherwise
print("Looking for "+str(column_count)+" terms on "+str(row_count)+" pages.") 

Looking for 2 terms on 2 pages.


In [52]:
# Go get a snapshot of the page
for pos, page in enumerate(pages):
    final_urls[page]=""
    try:
        with WaybackClient() as client:
            dump = client.search(page, from_date=datetime(dates[0], dates[1],dates[2]), to_date=datetime(dates[3], dates[4], dates[5])) # list_versions calls the CDX API from internetarchive.py from the webmonitoring repo
            versions = reversed(list(dump))
            for version in versions: # For each version in all the snapshots
                if version.status_code == 200 or version.status_code == '-': # If the IA snapshot was viable...
                    url=version.raw_url
                    response = client.get_memento(url)
                    content = response.content.decode()
                    for p, t in enumerate(terms):
                        page_sum = content.lower().count(t)
                        matrix[pos][p]=page_sum
                        print("Counted " + t + "" + str(page_sum) + " times on " + url)
                    final_urls[page]=url
                    break
                else:
                    pass
    except:
        print("No snapshot or can't decode", page)
        final_urls[page]=""
matrix[pos]=999

Counted climate11 times on http://web.archive.org/web/20160630065006id_/https://www3.epa.gov/
Counted climate change3 times on http://web.archive.org/web/20160630065006id_/https://www3.epa.gov/
Counted climate165 times on http://web.archive.org/web/20160627220851id_/https://www3.epa.gov/climatechange/
Counted climate change42 times on http://web.archive.org/web/20160627220851id_/https://www3.epa.gov/climatechange/


In [50]:
# Report results
for pos,term in enumerate(terms):
    for p, page in enumerate(pages):
        t = ""
        if type(term) == list:
            for part in term:
                t += part + " "
            t = t[:-1]
        else:
            t = term
        print("'"+ t +"': "+ str(matrix[p][pos]) +" on " + pages[p] + " ("+final_urls[pages[p]]+")")

'climate': 11 on epa.gov (http://web.archive.org/web/20160630065006id_/https://www3.epa.gov/)
'climate': 999 on epa.gov/climatechange (http://web.archive.org/web/20160627220851id_/https://www3.epa.gov/climatechange/)
'climate change': 3 on epa.gov (http://web.archive.org/web/20160630065006id_/https://www3.epa.gov/)
'climate change': 999 on epa.gov/climatechange (http://web.archive.org/web/20160627220851id_/https://www3.epa.gov/climatechange/)


In [46]:
matrix

array([[ 11,   3],
       [999, 999]], dtype=int16)

In [36]:
url

'http://web.archive.org/web/20160628160750id_/https://www3.epa.gov/'

In [37]:
content.lower().count('climate change')

3